In [ ]:
import os
import uuid
import json
import logging
from dotenv import load_dotenv

from dapr_agents import OpenAIChatClient
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from dapr_agents.tool import AgentTool

logging.basicConfig(level=logging.INFO)
load_dotenv()

In [ ]:
llm = OpenAIChatClient(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
)

def to_obj(x):
    return json.loads(x) if isinstance(x, str) else x

In [ ]:
# ---- MCP Jupyter server env (REMOTE) ----
NOTEBOOK_NAME = f"mcp_{uuid.uuid4().hex[:8]}.ipynb"

ENV = {
    **os.environ,
    "MCP_JUPYTER_SESSION_MODE": "server",
    "MCP_JUPYTER_BASE_URL": os.getenv("JUPYTER_BASE_URL"),
    "MCP_JUPYTER_TOKEN": os.getenv("JUPYTER_TOKEN"),
    "MCP_JUPYTER_KERNEL_NAME": os.getenv("JUPYTER_KERNEL_NAME", "python3"),
    "MCP_JUPYTER_NOTEBOOK_PATH": os.getenv("JUPYTER_NOTEBOOK_PATH", NOTEBOOK_NAME),
}

# Adjust path to your MCP server entrypoint (where your server.py lives)
server_params = StdioServerParameters(
    command="/Users/wardog/.local/bin/uv",
    args=["run", "mcp_example/server.py"],
    env=ENV,
)

In [ ]:
async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as s:
        await s.initialize()

        # Discover tools
        tools = {t.name: t for t in await AgentTool.from_mcp_session(s)}
        print("Discovered tools:", list(tools.keys()))

        # 1) Add a markdown heading
        md_res = await tools["Notebook.Markdown.Add"].arun(content="## Hello from MCP Jupyter")
        print("markdown.add ->", to_obj(md_res))

        # 2) Run a code cell
        code = "print('hello'); 1+2"
        run_res = await tools["Notebook.Code.Run"].arun(content=code)
        run_obj = to_obj(run_res)
        print("code.run ->", run_obj["ok"], run_obj.get("stdout"), run_obj.get("formatted_output"))